# Title

## Introduction

Introduction instructions:

provide some relevant background information on the topic so that someone 
unfamiliar with it will be prepared to understand the rest of your report
clearly state the question you tried to answer with your project
identify and fully describe the dataset that was used to answer the question

I added my data description below but it obviously needs a lot of editing to be the right format

### players_data Information
#### Dataframe
- 196 Observations
- 7 Variables
- Experience was collected from players themselves
- Each name is unique currently but that is not guaranteed
- Issue: Age has 2 N/A values
- Note: Age starts with a capital A
#### Variables
| Name | Type | Description |
| :------- | :------ | :-------
| experience | chr | Categorical (Pro, Veteran, Amateur, Regular, Beginner)  | 
| subscribe | lgl | Categorical (TRUE/FALSE) |
| hashedEmail | chr | Identifies players |
| played_hours | dbl | Numerical (Total amount of hours played) |
| name | chr | Identifies players |
| gender | chr | Categorical (Male, Female, Non-binary, Prefer not to say, Agender, Two-Spirited, Other) |
| Age | dbl | Numerical (Age of the player) |
#### Summary Statisitics
| Name | Unique Observations | Max | Min | Mean |
| :------- | :------ | :-------| :-------| :-------
| played_hours | 43 | 223.1 | 0 | 5.85 |
| Age | 33 | 58 | 9 | 21.14 |
| experience | 5 | - |  - | - | 
| subscribe | 2 | - | - | - | 
| hashedEmail | 196 | - | - | - |  
| name | 196 | - | - | - | 
| gender | 7 | - | - | - |  